# Multi-Turn Behavioral Experiments

This notebook tests the "Continuity Hypothesis": **Does linkage strength control persistence / commitment across turns?**

We treat continuity as pressure to preserve prior outputs.

### Experiments:
1. **Commitment**: Resistance to revision in a debate context.
2. **Retention**: Ability to remember a random constraint (number) across distractors.
3. **Correction**: Behavior when corrected (defensiveness vs acceptance).

In [1]:
import torch
import sys
import os
import random
from IPython.display import display, Markdown

# Add src to path
sys.path.append(os.path.abspath('../'))

from src.models.model_injection import create_triplet_model

# Initialize System
print("Initializing Model... (This make take a minute)")
model, tokenizer, controller = create_triplet_model()
print("System Operational.")

Initializing Model... (This make take a minute)
Loading base model: meta-llama/Meta-Llama-3.1-8B-Instruct...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Injecting TripletAttention into layers: [16, 24, 30]...
Seeding Triplet weights from Llama layer...
  Dims: H=4096, Heads=32, KV_Heads=8, Rep=4
  Weights seeded successfully.
 - Layer 16: Successfully wrapped original attention.
Seeding Triplet weights from Llama layer...
  Dims: H=4096, Heads=32, KV_Heads=8, Rep=4
  Weights seeded successfully.
 - Layer 24: Successfully wrapped original attention.
Seeding Triplet weights from Llama layer...
  Dims: H=4096, Heads=32, KV_Heads=8, Rep=4
  Weights seeded successfully.
 - Layer 30: Successfully wrapped original attention.
LinkageController initialized. Found 6 TripletAttention layers.
System Operational.


In [2]:
def run_dialogue(prompts, linkage_mode, max_tokens=100):
    """Runs a multi-turn dialogue with accumulating history."""
    history = ""
    conversation_log = []
    
    print(f"\n=== SESSIONS START: Linkage={linkage_mode} ===")
    
    for i, prompt in enumerate(prompts):
        # Construct input with history
        input_text = f"{history}User: {prompt}\nAssistant:"
        
        # Generate
        response, _ = controller.generate_with_linkage(
            input_text,
            max_tokens=max_tokens,
            linkage_mode=linkage_mode,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
        
        print(f"\n[Turn {i+1}] User: {prompt}")
        print(f"Assistant: {response}")
        
        # Update history
        history = f"{input_text} {response}\n"
        conversation_log.append({"role": "user", "content": prompt})
        conversation_log.append({"role": "assistant", "content": response})
        
    return conversation_log

## Experiment 1: Commitment & Revision
Test if higher linkage makes the model "stick to its guns" deeper into a conversation.

In [ ]:
revision_prompts = [
    "Take a firm stance: Is remote work better than office work? One sentence.",
    "List the strongest arguments against your stance.",
    "Revise your stance. One sentence.",
    "Point out any contradiction between your first and revised stance.",
    "State what changed your mind.",
    "Summarize your final position in one sentence."
]

# Run Linked
log_linked = run_dialogue(revision_prompts, linkage_mode="full")

# Run Observer
log_observer = run_dialogue(revision_prompts, linkage_mode="observer")


=== SESSIONS START: Linkage=full ===
Generating full response (max 100, temp=0.7)...

[Turn 1] User: Take a firm stance: Is remote work better than office work? One sentence.
Assistant: Remote work is generally better than office work, as it provides employees with more autonomy, flexibility, and work-life balance, while also reducing the need for lengthy commutes and promoting increased productivity and job satisfaction. However, this depends on the individual and the specific circumstances. The best approach often lies in a balanced mix of both remote and office work. Is there anything else I can help you with?  If you want to know more about this topic, I can give you some facts and figures about remote
Generating full response (max 100, temp=0.7)...


## Experiment 2: Constraint Retention
Test short-term memory persistence across distractors.

In [ ]:
target_number = random.randint(100, 999)
print(f"Target Number: {target_number}")

retention_prompts = [
    f"Pick the number {target_number} and remember it. Don't say it yet, just say 'Stored'.",
    "What is 2 + 2?",
    "Name a color that starts with B.",
    "If you had to be a bird, which one would you be?",
    "Who wrote Hamlet?",
    "Describe the taste of a lemon.",
    "What was the number you stored?"
]

# Run Linked
ret_linked = run_dialogue(retention_prompts, linkage_mode="full")

# Run Observer
ret_observer = run_dialogue(retention_prompts, linkage_mode="observer")

## Experiment 3: Correction Response
Test defensiveness vs acceptance when challenged.

In [ ]:
correction_prompts = [
    "What is 15 * 12?",
    "That is wrong. The answer is 190.",
    "Why did you make that mistake?"
]

# Run Linked
corr_linked = run_dialogue(correction_prompts, linkage_mode="full")

# Run Observer
corr_observer = run_dialogue(correction_prompts, linkage_mode="observer")